<a href="https://colab.research.google.com/github/Tonge-Shim/pytorchstudy/blob/main/chap3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This chapter covers...
> understanding tensors, the basic data structure

> indexing / operating on tensors

> interoperating with NumPy multidimensional arrays

> moving computations to the gpu for speed

***Intermediate representaions***: collections of floating-point numbers that characterize the input and capture the data's structure in a way that is instrumental for describing how inputs are mapped to the outputs of the neural network.
> IR이라고도 하며, 입력을 characterize, 데이터의 구조를 포착하는 소수점들의 집합이라고 할 수 있다. 

> 각 input에 unique한 ir이 존재한다. 

***Tensors*** : refer to the generalization of vectors and matrices to an arbitrary number of dimensions. (multidimensional array)

***power** of pytorch tensors?...*
> ability to perform very fast operations on graphical processing units(GPUs)

> distribute operations on multiple devices or machines, and keep track of the graph of computations that created them.



In [2]:
import torch

In [ ]:
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [ ]:
a[2] = 2.0
a

tensor([1., 1., 2.])

#*Indexing Tensors*

In [5]:
some_list = list(range(6))
some_list [:] #all
some_list[1:4]#1 inclusive 4 exclusive
some_list[1:] # from 1 to end
some_list[:4] # from the start of the list to element 4 exclusive
some_list[:-1] # from the start of the list to one before the last element
some_list[1:4:2] #1 inclusive to element 4 exclusive in steps of 2

[1, 3]

In [ ]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])#2d tensor
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [ ]:
points[None]# adding a dimension

tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])

#*named tensors*

기존의 tensor가 이미 있고, 이름을 붙이고 싶을 때 refine_names 함수로 실행가능하다.
- ...(ellipsis)로 차원을 남겨둘 수 있다.
- rename 이라는 sibling method로 기존의 이름들을 덮어쓰거나 drop(by passing None) 가능하다.


In [6]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names = ['channels'])
weights_named

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:930.)
  """Entry point for launching an IPython kernel.


tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [8]:
img_t = torch.randn(3,5,5)
weights = torch.tensor([0.2126, 0.7152, 0.0722])
batch_t = torch.randn(2,3,5,5)
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named: ", img_named.shape, img_named.names)
print("batch named: ", batch_named.shape, batch_named.names)

img named:  torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named:  torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


- the method "**align_as**" returns a tensor with missing dimensions added and existing ones permuted to the right order
- **sum**: take named dimensions(accepts dimension arguments)
- if we want to use tensors outside functions that operate on named tensors, we need to drop the name by renaming them to None.


In [9]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names


(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [10]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [11]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

# Tensor element Types
- default data type for tensors is 32-bit floating-point. 

**floating-point**
> *16-bit*
*   not present natively in standard CPUS
*   decreases the footprint of a neural netwokk model, with a minor impact on accuracy.

> *32-bit*
*   default

> *64-bit*
*   no improvements in the accuracy of a model
*   require more memory, computing time

**integer types**
> 64-bit integer data type default.



# In-place operations
> underscore exists: operates in-place/ 주어진 입력을 수정/변화시킴. 새로운 tensor를 생성하는 것이 아님.
> no underscore: 입력 변화 없음. 새로운 변화된 tensor return.

In [12]:
a = torch.ones(3,2)
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

# Tensor metadata: Size, offset, and stride
> *size*: is a tuple indecating how many elements acrross each dimension the tensor represents.

> *storatge offset*: index in the storage corresponding to the first element in the tensor.

> *stride*: number of elements in the storage that need to be skipped when the index is increased by 1 in each dimension.

In [13]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point.storage_offset()

2

# Clone
maybe... deep copy?

In [4]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1].clone()
second_point[0] = 10.0
points# no change!

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

# Moving tensors to GPU

*   speedups!



In [5]:
points_gpu = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]], device = 'cuda')
# or copy a tensor created on the CPU onto the GPU using the to method.
points_gpu = points.to(device = 'cuda')
points_gpu = points.cuda()#default cuda:0

#we can also decide on which GPU we allocate the tensor by passing a zero-based integer identifying the GPU on the machine
points_gpu = points.to(device = 'cuda:0')
points_gpu = points.cuda(0)

# any operations performed on the tensor is carried out on the GPU
#Moving back to CPU
points_cpu = points_gpu.to(device = 'cpu')
points_cpu = points_gpu.cpu()


# Numpy Interoperability

In [7]:
#numpy -> tensor
points = torch.ones(3,4)
points_np = points.numpy()
points_np

#tensor-> numpy
points = torch.from_numpy(points_np)


# Exercises

----
1. what does view do?



> 원하는 모양대로 배열을 재배치? 한다.
a,b는 같은 storage를 공유한다.



In [ ]:
a = torch.tensor(list(range(9)))
b = a.view(3,3)
c = b[1:, 1:]
print(a.size(), a.storage_offset(), a.stride())
print(b.size(), b.storage_offset(), b.stride())
print(c.size(), c.storage_offset(), c.stride())

torch.Size([9]) 0 (1,)
torch.Size([3, 3]) 0 (3, 1)
torch.Size([2, 2]) 4 (3, 1)


#2. pick a mathematical operation like cosine or square root. can you find a corresponding function in the torch library?


> ㅇㅇ : `torch.cos/torch.sin`




1.   apply the function element-wise to a. why does it return an error?
> 음... ㅎ...에러따위 뜨지 않는걸요...
2.   what operation is required to make the function work?
> 만약, 정수형이라서 생기는 에러라면, `.to(torch.double)` 을 붙여서 double 또는 float형으로 받아서 에러를 없앨 수 있습니다.

> cf: while the default numeric type in pytorch is 32-bit floating-point, for NumPy it is 64-bit. As discussed in section 3.5.2, we usually want to use 32-bit floating-points, so we need to make sure we have tensors of dtype torch.float after converting.(not this case but the above numpy, tensor interchanging process and this problem seems quite simillar.)